In [62]:

import cvxpy as cp

V = [0, 1, 2, 3]
E = [(0,1),(1,2),(2,3)]
R = [1]
p = {0:0.0, 1:2.0, 2:3.0, 3:0.0}
b_i_k = {(1,1):5.0, (2,1):5.0}
b_k = {1:10.0}
M = 1e5

source = 0
sink = 3

# ----- variables -----
x = {(i,j): cp.Variable(boolean=True, name=f"x_{i}_{j}") for i in V for j in V if i != j}
S = {i: cp.Variable(nonneg=True, name=f"S_{i}") for i in V}
rho = cp.Variable(nonneg=True, name="rho")
f = {(i,j,k): cp.Variable(nonneg=True, name=f"f_{i}_{j}_{k}") for i in V for j in V if i != j for k in R}

# ----- constraints -----
constraints = []

# (1)
constraints += [rho >= S[sink]]

# (2) sequencing
for i in V:
    for j in V:
        if i == j:
            continue
        constraints += [S[j] >= S[i] + p.get(i, 0.0) - M * (1 - x[i,j])]

# (4) flow conservation
for i in V:
    if i in (source, sink):
        continue
    for k in R:
        inflow = cp.sum([f[j,i,k] for j in V if j != i])
        outflow = cp.sum([f[i,j,k] for j in V if j != i])
        demand = b_i_k.get((i,k), 0.0)
        constraints += [inflow == demand, outflow == demand]

# (5) source/sink totals
for k in R:
    constraints += [cp.sum([f[source,j,k] for j in V if j != source]) == b_k[k]]
    constraints += [cp.sum([f[j,sink,k] for j in V if j != sink]) == b_k[k]]

# (7) capacity
for i in V:
    for j in V:
        if i == j:
            continue
        for k in R:
            constraints += [f[i,j,k] <= M * x[i,j]]

# (8) force edges
for (i,j) in E:
    constraints += [x[i,j] == 1]

# objective and solve
obj = cp.Minimize(rho)
prob = cp.Problem(obj, constraints)

prob.solve(solver="GUROBI")

print("Status:", prob.status)
print("rho:", rho.value)
print("S:", {i: int(S[i].value) for i in V})
print("x :", {k: int(v.value) for k,v in x.items()})
print("f (nonzero):", {k: int(v.value) for k,v in f.items() if (v.value is not None and v.value > 1e-9)})


Status: optimal
rho: 5.0
S: {0: 0, 1: 0, 2: 2, 3: 5}
x : {(0, 1): 1, (0, 2): 0, (0, 3): 1, (1, 0): 0, (1, 2): 1, (1, 3): 0, (2, 0): 0, (2, 1): 0, (2, 3): 1, (3, 0): 0, (3, 1): 0, (3, 2): 0}
f (nonzero): {(0, 1, 1): 5, (0, 3, 1): 5, (1, 2, 1): 5, (2, 3, 1): 5}


**GPT TRY 1**

In [1]:
import cvxpy as cp

# ========================
# --- INPUT DEFINITION ---
# ========================

data = {
    "V": [0, 1, 2, 3],                # צמתים
    "E": [(0, 1), (1, 2), (2, 3)],   # קשתות
    "R": [1],                         # קבוצת בקשות / סוגי זרמים
    "source": 0,
    "sink": 3,
    "p": {0: 0.0, 1: 2.0, 2: 3.0, 3: 0.0},    # זמני עיבוד או משקלים
    "b_i_k": {(1, 1): 5.0, (2, 1): 5.0},      # ביקוש מקומי לכל קודקוד ומשאב
    "b_k": {1: 10.0},                         # ביקוש כולל לכל משאב
    "M": 1e5,                                 # קבוע גדול
}

V = data["V"]
E = data["E"]
R = data["R"]
source = data["source"]
sink = data["sink"]
p = data["p"]
b_i_k = data["b_i_k"]
b_k = data["b_k"]
M = data["M"]

# =========================
# --- VARIABLE CREATION ---
# =========================

x = {(i, j): cp.Variable(boolean=True, name=f"x_{i}_{j}") for i in V for j in V if i != j}
S = {i: cp.Variable(nonneg=True, name=f"S_{i}") for i in V}
rho = cp.Variable(nonneg=True, name="rho")
f = {(i, j, k): cp.Variable(nonneg=True, name=f"f_{i}_{j}_{k}") for i in V for j in V if i != j for k in R}

# ======================
# --- CONSTRAINTS ------
# ======================

constraints = []

# (1) Objective upper bound
constraints += [rho >= S[sink]]

# (2) Sequencing constraints
for i in V:
    for j in V:
        if i == j:
            continue
        constraints += [S[j] >= S[i] + p.get(i, 0.0) - M * (1 - x[i, j])]

# (4) Flow conservation
for i in V:
    if i in (source, sink):
        continue
    for k in R:
        inflow = cp.sum([f[j, i, k] for j in V if j != i])
        outflow = cp.sum([f[i, j, k] for j in V if j != i])
        demand = b_i_k.get((i, k), 0.0)
        constraints += [inflow == demand, outflow == demand]

# (5) Total flow for source/sink
for k in R:
    constraints += [
        cp.sum([f[source, j, k] for j in V if j != source]) == b_k[k],
        cp.sum([f[j, sink, k] for j in V if j != sink]) == b_k[k],
    ]

# (7) Capacity
for i in V:
    for j in V:
        if i == j:
            continue
        for k in R:
            constraints += [f[i, j, k] <= M * x[i, j]]

# (8) Force edges that exist
for (i, j) in E:
    constraints += [x[i, j] == 1]

# =======================
# --- OBJECTIVE + SOLVE -
# =======================

obj = cp.Minimize(rho)
prob = cp.Problem(obj, constraints)
prob.solve(solver="GUROBI")

# =======================
# --- RESULTS ------------
# =======================

print("Status:", prob.status)
print("rho:", rho.value)
print("S:", {i: round(S[i].value, 3) for i in V})
print("x :", {k: int(v.value) for k, v in x.items()})
print("f (nonzero):", {k: round(v.value, 2) for k, v in f.items() if v.value and v.value > 1e-9})


Restricted license - for non-production use only - expires 2026-11-23
Status: optimal
rho: 5.0
S: {0: np.float64(0.0), 1: np.float64(0.0), 2: np.float64(2.0), 3: np.float64(5.0)}
x : {(0, 1): 1, (0, 2): 0, (0, 3): 1, (1, 0): 0, (1, 2): 1, (1, 3): 0, (2, 0): 0, (2, 1): 0, (2, 3): 1, (3, 0): 0, (3, 1): 0, (3, 2): 0}
f (nonzero): {(0, 1, 1): np.float64(5.0), (0, 3, 1): np.float64(5.0), (1, 2, 1): np.float64(5.0), (2, 3, 1): np.float64(5.0)}


**WITH RANDOM ELEMENTS**

In [5]:
import cvxpy as cp
import random

# ============================
# --- RANDOM PROBLEM GENERATION ---
# ============================

def generate_feasible_problem(
    n_nodes=6,
    n_resources=2,
    p_range=(1, 5),
    b_range=(5, 10),
    edge_prob=0.6,
    seed=42
):
    """
    יוצר בעיית זרימה/תזמון רנדומלית שהיא ברת פתרון.
    יוצר גרף DAG (כיוונים קדימה בלבד) וביקושים עקביים.
    """
    random.seed(seed)
    V = list(range(n_nodes))
    R = list(range(1, n_resources + 1))
    source, sink = 0, n_nodes - 1

    # ניצור גרף DAG (כל קשת i->j רק אם i<j)
    E = [(i, j) for i in V for j in V if i < j and random.random() < edge_prob]

    # לוודא שיש לפחות מסלול תקף
    if not any(j == sink for (i, j) in E):
        E.append((n_nodes - 2, sink))
    if not any(i == source for (i, j) in E):
        E.append((source, 1))

    # זמני עיבוד רנדומליים לכל קודקוד
    p = {i: round(random.uniform(*p_range), 2) for i in V}

    # ביקושים לכל (i,k) – רק לצמתים פנימיים בחלק מהמקרים
    b_i_k = {}
    for i in V:
        if i not in (source, sink) and random.random() < 0.5:
            for k in R:
                b_i_k[(i, k)] = round(random.uniform(*b_range), 2)

    # ביקושים כוללים (b_k) לפי סך הביקושים הפנימיים
    b_k = {k: sum(v for (i, kk), v in b_i_k.items() if kk == k) for k in R}

    # אם לא נוצר ביקוש פנימי – נוודא שיש ביקוש חיובי
    for k in R:
        if b_k[k] == 0:
            b_k[k] = round(random.uniform(*b_range), 2)

    M = 1e4

    data = dict(V=V, E=E, R=R, p=p, b_i_k=b_i_k, b_k=b_k,
                source=source, sink=sink, M=M)
    return data


# ===========================
# --- MODEL CONSTRUCTION ----
# ===========================

def solve_problem(data):
    """
    בונה את המודל לפי הנתונים, פותר ומחזיר את התוצאה.
    """
    V, E, R = data["V"], data["E"], data["R"]
    p, b_i_k, b_k = data["p"], data["b_i_k"], data["b_k"]
    source, sink, M = data["source"], data["sink"], data["M"]

    # משתנים
    x = {(i, j): cp.Variable(boolean=True, name=f"x_{i}_{j}") for i in V for j in V if i != j}
    S = {i: cp.Variable(nonneg=True, name=f"S_{i}") for i in V}
    rho = cp.Variable(nonneg=True, name="rho")
    f = {(i, j, k): cp.Variable(nonneg=True, name=f"f_{i}_{j}_{k}")
         for i in V for j in V if i != j for k in R}

    constraints = []

    # (1) מגבלת rho
    constraints += [rho >= S[sink]]

    # (2) אילוצי סדר
    for i in V:
        for j in V:
            if i == j:
                continue
            constraints += [S[j] >= S[i] + p[i] - M * (1 - x[i, j])]

    # (4) שימור זרימה
    for i in V:
        if i in (source, sink):
            continue
        for k in R:
            inflow = cp.sum([f[j, i, k] for j in V if j != i])
            outflow = cp.sum([f[i, j, k] for j in V if j != i])
            demand = b_i_k.get((i, k), 0.0)
            constraints += [inflow == demand, outflow == demand]

    # (5) זרימה כוללת ב-source וב-sink
    for k in R:
        constraints += [
            cp.sum([f[source, j, k] for j in V if j != source]) == b_k[k],
            cp.sum([f[j, sink, k] for j in V if j != sink]) == b_k[k],
        ]

    # (7) קיבולת
    for i in V:
        for j in V:
            if i == j:
                continue
            for k in R:
                constraints += [f[i, j, k] <= M * x[i, j]]

    # (8) להכריח קשתות שקיימות
    for (i, j) in E:
        constraints += [x[i, j] == 1]

    # פונקציית מטרה
    obj = cp.Minimize(rho)
    prob = cp.Problem(obj, constraints)

    prob.solve(solver="GUROBI", verbose=False)
    return prob, (x, S, rho, f)


# ==========================
# --- RUN EXAMPLE ----------
# ==========================

if __name__ == "__main__":
    data = generate_feasible_problem(
        n_nodes=6,
        n_resources=2,
        p_range=(1, 4),
        b_range=(5, 10),
        edge_prob=0.6,
        seed=10
    )

    prob, (x, S, rho, f) = solve_problem(data)

    print("Status:", prob.status)
    if prob.status not in ["infeasible", "unbounded"]:
        print("rho:", round(rho.value, 3))
        print("S:", {i: round(S[i].value, 2) for i in data["V"]})
        print("E:", data["E"])
        print("b_i_k:", data["b_i_k"])
        print("b_k:", data["b_k"])
        print("x active:", {k: int(v.value) for k, v in x.items() if v.value > 0.5})
        print("f nonzero:", {k: round(v.value, 2) for k, v in f.items() if v.value and v.value > 1e-6})
    else:
        print("Problem infeasible or unbounded.")


Status: optimal
rho: 7.68
S: {0: np.float64(0.0), 1: np.float64(2.81), 2: np.float64(2.81), 3: np.float64(4.66), 4: np.float64(4.95), 5: np.float64(7.68)}
E: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (2, 3), (2, 4), (3, 5)]
b_i_k: {(2, 1): 8.84, (2, 2): 9.91}
b_k: {1: 8.84, 2: 9.91}
x active: {(0, 1): 1, (0, 2): 1, (0, 3): 1, (0, 4): 1, (1, 4): 1, (1, 5): 1, (2, 3): 1, (2, 4): 1, (2, 5): 1, (3, 5): 1}
f nonzero: {(0, 2, 1): np.float64(8.84), (0, 2, 2): np.float64(9.91), (2, 5, 1): np.float64(8.84), (2, 5, 2): np.float64(9.91)}


***budget advarsrial***

In [5]:
import cvxpy as cp
import random


# === MANUAL PARAMETERS ===

# Node set: source = 0, sink = 4
V = [0, 1, 2, 3, 4]

# Edge set E only
E = [
    (0, 1), (0, 2),
    (1, 2), (1, 3),
    (2, 3), (2, 4),
    (3, 4)
]

# Parameters bar_p and hat_p
bar_p = {i: round(random.uniform(0.5, 3.0), 2) for i in V}
hat_p = {i: round(random.uniform(0.1, 1.5), 2) for i in V}

# Set source and sink (first and last) to zero
bar_p[V[0]] = 0.0
bar_p[V[-1]] = 0.0
hat_p[V[0]] = 0.0
hat_p[V[-1]] = 0.0

# Thresholds Π = {0, hat_p1, ..., hat_pn}
Pi = [0.0] + sorted(set(hat_p.values()))

source = 0
sink = 4


# === HELPER ===
def pos(x):
    return max(x, 0.0)


# === MAIN LOOP OVER THRESHOLDS ===
for pi in Pi:
    print(f"\n=== Threshold π = {pi} ===")

    # Arc weights: w_ij = -(bar_p_i + [hat_p_i - π]^+)
    w = {(i, j): -(bar_p[i] + pos(hat_p[i] - pi)) for (i, j) in E}

    # Decision variables φ_ij ∈ {0,1}
    phi = {(i, j): cp.Variable(boolean=True, name=f"phi_{i}_{j}") for (i, j) in E}

    constraints = []

    # (1) Flow conservation for intermediate nodes
    for v in V:
        if v in [source, sink]:
            continue
        inflow = cp.sum([phi[(i, j)] for (i, j) in E if j == v])
        outflow = cp.sum([phi[(i, j)] for (i, j) in E if i == v])
        constraints.append(inflow - outflow == 0)

    # (2) Source constraint
    constraints.append(cp.sum([phi[(i, j)] for (i, j) in E if i == source]) == 1)

    # (3) Sink constraint
    constraints.append(cp.sum([phi[(i, j)] for (i, j) in E if j == sink]) == 1)

    # Objective function
    objective = cp.Minimize(cp.sum([w[(i, j)] * phi[(i, j)] for (i, j) in E]))

    # Solve
    prob = cp.Problem(objective, constraints)
    prob.solve(solver="GUROBI", verbose=False)

    print("Status:", prob.status)
    if prob.status not in ["infeasible", "unbounded"]:
        print("Objective value:", round(prob.value, 3))
        selected_edges = [(i, j) for (i, j) in E if phi[(i, j)].value > 0.5]
        print("Selected edges:", selected_edges)
    else:
        print("No feasible solution.")



=== Threshold π = 0.0 ===
Status: optimal
Objective value: -8.98
Selected edges: [(0, 1), (1, 2), (2, 3), (3, 4)]

=== Threshold π = 0.0 ===
Status: optimal
Objective value: -8.98
Selected edges: [(0, 1), (1, 2), (2, 3), (3, 4)]

=== Threshold π = 0.83 ===
Status: optimal
Objective value: -6.49
Selected edges: [(0, 1), (1, 2), (2, 3), (3, 4)]

=== Threshold π = 1.49 ===
Status: optimal
Objective value: -5.83
Selected edges: [(0, 1), (1, 2), (2, 3), (3, 4)]
